In [ ]:
import utils as ui ##needed to import my iris
import iris
import numpy as np

In [ ]:
def fill_array(tc,xc,yc,zc):
    valmax = 30
    valmin = 0

    t = tc.points
    x = xc.points
    y = yc.points
    z = zc.points
    temp = (valmax-valmin) * np.einsum('i,j,k,l',1+np.sin((t/8760.)/(2.*np.pi)), 1+np.sin(x), np.cos(y), (z - zmax)/(zmax-zmin))

    return temp

In [ ]:
tmin = 0
tmax = 11*8760 # 8760 ~ 1yr
tsamp = 100

xmin = -179
xmax = 179
xsamp = 358

ymin = -89
ymax = 89
ysamp = 178

zmin = 0
zmax = 7000
zsamp = 7

#Create coordinates:
tco = iris.coords.DimCoord(np.linspace(tmin,tmax,tsamp), standard_name='time', units='hours since 2000-01-01 00:00')
tco.guess_bounds()

xco = iris.coords.DimCoord(np.linspace(xmin,xmax,xsamp), standard_name='longitude', units='degree')
xco.guess_bounds()

yco = iris.coords.DimCoord(np.linspace(ymin,ymax,ysamp), standard_name='latitude', units='degree')
yco.guess_bounds()

zco = iris.coords.DimCoord(np.linspace(zmin,zmax,zsamp), standard_name='height', units='metres')
zco.guess_bounds()

#Create cube:
c1 = iris.cube.Cube(fill_array(tco,xco,yco,zco), standard_name='air_temperature', units='celsius')
c1.add_dim_coord(tco, 0)
c1.add_dim_coord(xco, 1)
c1.add_dim_coord(yco, 2)
c1.add_dim_coord(zco, 3)

In [ ]:
#Adapted from http://scitools.org.uk/iris/docs/latest/examples/General/anomaly_log_colouring.html
import cartopy.crs as ccrs
import iris.coord_categorisation
import matplotlib.pyplot as plt
import iris.plot as iplt
import iris.quickplot as qplt

# Enable a future option, to ensure that the netcdf load works the same way
# as in future Iris versions.
iris.FUTURE.netcdf_promote = True

# Load a sample air temperatures sequence.
temperatures = c1

# Create a year-number coordinate from the time information.
try:
    #If this has already beendon, iris will complain.
    #So we ignore the complaint.
    iris.coord_categorisation.add_year(temperatures, 'time')
except:
    pass

# Create a sample anomaly field for one chosen year, by extracting that
# year and subtracting the time mean.
sample_year = 2001
year_temperature3d = temperatures.extract(iris.Constraint(year=sample_year))
year_temperature = year_temperature3d.collapsed(['time','height'], iris.analysis.MEAN)
time_mean = temperatures.collapsed(['time','height'], iris.analysis.MEAN)
anomaly = (year_temperature - time_mean)


# Construct a plot title string explaining which years are involved.
years = temperatures.coord('year').points
plot_title = 'Temperature anomaly'
plot_title += '\n{} differences from {}-{} average.'.format(
        sample_year, years[0], years[-1])

    
# Define scaling levels for the logarithmic colouring.
minimum_level = 0.0
maximum_level = 30

# Create an Axes, specifying the map projection.
plt.axes(projection=ccrs.Mollweide())
#plt.axes(projection=ccrs.Stereographic())

# Make a pseudocolour plot using this colour scheme.
# Use a standard colour map which varies blue-white-red.
# For suitable options, see the 'Diverging colormaps' section in:
# http://matplotlib.org/examples/color/colormaps_reference.html
anom_cmap = 'bwr'
mesh = iplt.pcolormesh(anomaly, cmap=anom_cmap)

# Add a colourbar, with extensions to show handling of out-of-range values.
#bar = plt.colorbar(mesh, orientation='horizontal', extend='both')

# Set some suitable fixed "logarithmic" colourbar tick positions.
#tick_levels = [-3, -1, -0.3, 0.0, 0.3, 1, 3]
#bar.set_ticks(tick_levels)

# Modify the tick labels so that the centre one shows "+/-<minumum-level>".
#tick_levels[3] = r'$\pm${:g}'.format(minimum_level)
#bar.set_ticklabels(tick_levels)

# Label the colourbar to show the units.
#bar.set_label('[{}]'.format(anomaly.units))

# Add coastlines and a title.
plt.gca().coastlines()
plt.title(plot_title)

# Display the result.
iplt.show()
